In [1]:
import torch
from mamba_ssm import Mamba2

/home/tzeng/anaconda3/envs/dai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = Mamba2(
    d_model=16,   # hidden size
    d_state=16,   # state size
    d_conv=4,     # convolution size
    expand=2      # expansion factor
)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Dummy input: batch of 2 sequences, each length 8, embedding dim = 16
x = torch.randn(2, 8, 16).to(device)

# Forward pass
y = model(x)

print("Torch:", torch.__version__, "CUDA:", torch.version.cuda)
print("CUDA available?", torch.cuda.is_available())
print("Torch path:", torch.__file__)
print("Input shape:", x.shape)
print("Output shape:", y.shape)

AssertionError: 